In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
from qiskit.circuit.library import HGate
import math

In [3]:
# Setup operators
s2, rs2 = math.sqrt(2), 1 / math.sqrt(2)
rd1, rd2 = 1 / math.sqrt(4 + 2*s2), 1 / math.sqrt(4 - 2*s2)

W = Operator([
    [-1 * rd1, (1 + s2) * rd1],
    [rd2, (s2 - 1) * rd2]
])

V = Operator([
    [rd1, (1 + s2) * rd1],
    [-1 * rd2, (s2 - 1) * rd2]
])

rs3 = 1 / math.sqrt(3)
third = Operator([
    [rs3, rs3 * -1 * s2],
    [rs3 * s2, rs3]
])

In [4]:
def random_third() -> int:
    circuit = QuantumCircuit(2)
    circuit.append(third, [0])  # Apply one third unitary to first qubit
    circuit.h(1)  # Use Hadamard on second qubit as tie-breaker
    circuit.measure_all()
    
    backend = BasicSimulator()
    compiled = transpile(circuit, backend)
    sim = backend.run(compiled, shots=1)
    counts = sim.result().get_counts()
    
    q2, q1 = list(counts.keys())[0]
    if q1 == "0":
        return 0
    if q2 == "0":
        return 1
    return 2

def test_random_third() -> None:
    counts = {x: 0 for x in range(3)}
    for _ in range(1000):
        counts[random_third()] += 1
    print(counts)
# test_random_third()

In [5]:
# Constants
ALICE_OPS = ("X", "W", "Z")
BOB_OPS = ("W", "Z", "V")
OP_SETS = {
    "XW": "test", "XZ": "discard", "XV": "test",
    "WW": "key", "WZ": "discard", "WV": "discard",
    "ZW": "test", "ZZ": "key", "ZV": "test"
}

ENTANGLED_THRESHHOLD: float = 0.9 * 2 * s2

In [6]:
# Validation funcs
def validate_entanglement(abits: list[int], bbits: list[int], ops: list[str]) -> str:
    sums = {op_pair: 0 for op_pair, purpose in OP_SETS.items() if purpose == "test"}
    counts = sums.copy()
    
    for a, b, op in zip(abits, bbits, ops):
        if OP_SETS.get(op) == "test":
            a = -1 if a == 1 else 1
            b = -1 if b == 1 else 1
            sums[op] += a * b
            counts[op] += 1
    score = abs(
        sums["XW"] / counts["XW"] - sums["XV"] / counts["XV"] + sums["ZW"] / counts["ZW"] + sums["ZV"] / counts["ZV"]
    )
    if score > ENTANGLED_THRESHHOLD:
        return f"Entanglement validated (score: {score})."
    else:
        return f"Entanglement failed validation! (score: {score})."

def validate_key(abits: list[int], bbits: list[int], ops: list[str]) -> str:
    key_bits = []
    for a, b, op in zip(abits, bbits, ops):
        if OP_SETS.get(op) == "key":
            b = 0 if b == 1 else 1  # Invert Bob's bits for the key
            key_bits.append(str(a))
            if a != b:
                return "Key failed validation!"
    return f"Valid key generated: {''.join(key_bits)} (len: {len(key_bits)})."

In [21]:
from random import randrange

# Stuff but attacked
def ekert91(target_len: int, attack_prob_perc: int = 10) -> None:
    N = math.ceil(9 * target_len / 2)

    alice_operators = [ALICE_OPS[random_third()] for _ in range(N)]
    bob_operators = [BOB_OPS[random_third()] for _ in range(N)]
    op_pairs = [aop + bop for aop, bop in zip(alice_operators, bob_operators)]
    
    alice_bits = []
    bob_bits = []
    for op_pair in op_pairs:
        aop, bop = op_pair
    
        pair_circuit = QuantumCircuit(2, 1)  # Add a classical register for measuring during eavesdropping

        # Set initial state to |11>
        pair_circuit.x(0)
        pair_circuit.x(1)

        # Create entangled state rs2(|01> - |10>)
        pair_circuit.h(0)
        pair_circuit.cx(0, 1)

        # Simulate attack on some of Alice's qubits via measurement
        attacked = False
        if randrange(0, 100) < attack_prob_perc:
            attacked = True
            pair_circuit.measure(0, [0])

        # Apply Alice's selected operator to first qubit
        match aop:
            case "X":
                pair_circuit.h(0)
            case "W":
                pair_circuit.append(W, [0])
            case "Z":
                pass
            case _:
                raise Exception("!")

        # Apply Bob's selected operator to second qubit
        match bop:
            case "W":
                pair_circuit.append(W, [1])
            case "Z":
                pass
            case "V":
                pair_circuit.append(V, [1])
            case _:
                raise Exception("!")
    
        pair_circuit.measure_all()
        backend = BasicSimulator()
        compiled = transpile(pair_circuit, backend)
        sim = backend.run(compiled, shots=1)
        counts = sim.result().get_counts()

        bbit, abit, _, abit_measured = list(counts.keys())[0]
        if attacked:
            abit = abit_measured  # Alice's qubit has been measured, so she receives the result
            
        alice_bits.append(int(abit))
        bob_bits.append(int(bbit))
        
    print(validate_key(alice_bits, bob_bits, op_pairs))
    print(validate_entanglement(alice_bits, bob_bits, op_pairs))
# ekert91(16)

Key failed validation!
Entanglement failed validation! (score: 2.1933621933621934).


In [22]:
for trial in range(0, 10):
    print(f"\n---------------------- TRIAL {trial} ({trial * 10}% attack chance) ----------------------")
    ekert91(64, attack_prob_perc=trial * 10)


---------------------- TRIAL 0 (0% attack chance) ----------------------
Valid key generated: 011111110000010011101001011001000111000100001100111101011101111010111 (len: 69).
Entanglement validated (score: 3.271009182315863).

---------------------- TRIAL 1 (10% attack chance) ----------------------
Key failed validation!
Entanglement failed validation! (score: 2.496456118407338).

---------------------- TRIAL 2 (20% attack chance) ----------------------
Key failed validation!
Entanglement failed validation! (score: 2.3518255937610775).

---------------------- TRIAL 3 (30% attack chance) ----------------------
Key failed validation!
Entanglement failed validation! (score: 2.135125448028674).

---------------------- TRIAL 4 (40% attack chance) ----------------------
Key failed validation!
Entanglement failed validation! (score: 2.2915248404903577).

---------------------- TRIAL 5 (50% attack chance) ----------------------
Key failed validation!
Entanglement failed validation! (score: 2